In [1]:
import requests, runpod, os, config
os.environ['RUNPOD_API_KEY'] = config.RUNPOD_API_KEY

c:\Users\jesus\anaconda3\envs\runpod_endpoint\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
runpod.api_key = os.getenv("RUNPOD_API_KEY")

In [31]:
# Mistral Model
pod = runpod.create_pod(
    name='mistral-7b-chat',
    image_name='ghcr.io/huggingface/text-generation-inference:latest',
    gpu_type_id='NVIDIA GeForce RTX 4090',
    data_center_id='EU-RO-1',
    cloud_type='SECURE',
    docker_args= '--model-id TheBloke/Kimiko-Mistral-7B-fp16 --quantize eetq',
    gpu_count=1,
    volume_in_gb=80,
    container_disk_in_gb=15,
    ports='80/http,29500/http',
    volume_mount_path='/data',
)

In [51]:
pod

{'id': 'xuqocmk4f2ve75',
 'imageName': 'ghcr.io/huggingface/text-generation-inference:latest',
 'env': [],
 'machineId': 'f7aoknqdit15',
 'machine': {'podHostId': 'xuqocmk4f2ve75-64410f80'}}

In [54]:
server = f"https://{pod['id']}-80.proxy.runpod.net"
server

'https://xuqocmk4f2ve75-80.proxy.runpod.net'

In [55]:
def generate_prompt(prompt):
    return  f"""
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:

""".strip()

In [56]:
def make_request(prompt:str):
    data = {
        "inputs": prompt,
        "parameters": {"best_of": 1, "temperature":0.01, "max_new_tokens": 512},
    }
    headers = {
        "Content-Type": "application/json",
    }

    return requests.post(f"{server}/generate", json=data, headers=headers)

In [59]:
%%time
prompt = generate_prompt('I would like to understand more about the struggles of the 37th president of the US')
response = make_request(prompt)

CPU times: total: 219 ms
Wall time: 6.92 s


In [60]:
answer = response.json()['generated_text'].strip()
answer

'Richard Nixon was the 37th president of the United States, serving from 1969 to 1974. He faced many challenges during his presidency, including the Vietnam War, the Watergate scandal, and the economic recession of the early 1970s.\n\nThe Vietnam War was a major issue during Nixon\'s presidency. He inherited the war from his predecessor, Lyndon Johnson, and tried to find a way to end the conflict. Nixon implemented a policy of "Vietnamization," which involved gradually transferring responsibility for the war to the South Vietnamese government and military. However, the war dragged on for several more years, and Nixon\'s efforts to end it were ultimately unsuccessful.\n\nThe Watergate scandal was another major challenge for Nixon. In 1972, five men were caught breaking into the Democratic National Committee headquarters at the Watergate complex in Washington, D.C. It was later revealed that the break-in was part of a larger effort by Nixon\'s reelection campaign to spy on the Democrats.

In [8]:
# WizardModel
wizard_pod = runpod.create_pod(
    name='WizardLM-13B-chat',
    image_name='ghcr.io/huggingface/text-generation-inference:latest',
    gpu_type_id='NVIDIA GeForce RTX 4090',
    data_center_id='EU-RO-1',
    cloud_type='SECURE',
    docker_args= '--model-id WizardLM/WizardLM-13B-V1.2 --quantize eetq',
    gpu_count=1,
    volume_in_gb=80,
    container_disk_in_gb=10,
    ports='80/http,29500/http',
    volume_mount_path='/data',
)

In [9]:
wizard_pod

{'id': 'ldrenteuvo7rdk',
 'imageName': 'ghcr.io/huggingface/text-generation-inference:latest',
 'env': [],
 'machineId': '46gnla8ht5yz',
 'machine': {'podHostId': 'ldrenteuvo7rdk-64410faf'}}

In [74]:
runpod.terminate_pod(pod['id'])

In [14]:
server = f"https://{wizard_pod['id']}-80.proxy.runpod.net"
server

'https://ldrenteuvo7rdk-80.proxy.runpod.net'

In [16]:
server_docs = f"https://{wizard_pod['id']}-80.proxy.runpod.net/docs"
server_docs

'https://ldrenteuvo7rdk-80.proxy.runpod.net/docs'

In [11]:
default_system_prompt = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()

def generate_prompt(prompt: str, system_prompt: str = default_system_prompt) -> str:
    return f"""
    [INST] <<SYS>>
    {system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [48]:
def generate_prompt(prompt):
    return  f"""
A chat between a curious user and an artificial intelligence assistant. 
The assistant gives helpful, detailed, and polite answers to the user's 
questions. USER: {prompt} ASSISTANT: .</s>
""".strip()


In [18]:
def make_request(prompt:str):
    data = {
        "inputs": prompt,
        "parameters": {"best_of": 1, "temperature":0.01, "max_new_tokens": 512},
    }
    headers = {
        "Content-Type": "application/json",
    }

    return requests.post(f"{server}/generate", json=data, headers=headers)

In [49]:
%%time
prompt = generate_prompt('I would like to understand more about the struggles of the 37th president of the US')
response = make_request(prompt)

CPU times: total: 78.1 ms
Wall time: 7.25 s


In [50]:
answer = response.json()['generated_text'].strip()
answer 

'Richard Nixon was the 37th president of the United States, serving from 1969 to 1974. His presidency was marked by both significant achievements and major controversies.\n\nNixon\'s accomplishments included:\n\n• The Apollo 11 moon landing in 1969, which occurred during his first year in office.\n\n• The establishment of the Environmental Protection Agency (EPA) in 1970 to protect human health and the environment.\n\n• The signing of the Occupational Safety and Health Act in 1970 to improve workplace safety.\n\n• The signing of the Endangered Species Act in 1973 to protect threatened and endangered species.\n\n• The signing of the Clean Water Act in 1972 to improve water quality.\n\nHowever, Nixon\'s presidency was also marred by several major controversies:\n\n• The Watergate scandal, which began in 1972 when burglars were caught breaking into the Democratic National Committee headquarters at the Watergate complex in Washington, D.C. The scandal eventually led to Nixon\'s resignation

In [ ]:
# Stop pods
